# 测试控制台

你知道吗？阳见惠凪太可爱了

# V2版本控制台

## POD 构建（1级产物）

In [ ]:
from backend.pipelines.pod import quick_build_pod

res = quick_build_pod(
    nc_path="data/cylinder2d.nc",
    r=128,
    center=True,
    var_keys=("u", "v"),
    verbose=True,   # 打印所有中间过程
    plot=True,      # 画出奇异值谱+累计能量图
)

print("实际使用的模态数 r_used =", res["r_used"])
print("前10阶累计能量:", res["cum_energy"][:10])


## 训练主流程（2级产物）

In [ ]:
from backend.pipelines.train import compute_level2_rebuild

res = compute_level2_rebuild(
    "configs/cylinder_exp_full.yaml",
    experiment_name="cylinder_exp_full",
    save_root="artifacts/experiments",
    verbose=True,
)
print(res["exp_dir"])

In [ ]:
from backend.eval.rebuild import summarize_l2_npz
info = summarize_l2_npz("artifacts/experiments/cylinder_exp_small/L2/linear/linear_p0004_s0000.npz")
info


In [ ]:
from backend.eval.rebuild import plot_l2_coeffs_preview
plot_l2_coeffs_preview("artifacts/experiments/cylinder_exp_small/L2/linear/linear_p0004_s0000.npz", coeff_idx=3)

## 傅里叶初步分析（3级产物）

In [ ]:
from backend.pipelines.train import compute_level3_fft_from_level2

res = compute_level3_fft_from_level2(
    exp_dir="artifacts/experiments/cylinder_exp_full",
    verbose=True,
)

In [ ]:
from pathlib import Path
from backend.eval.pre_analysis import summarize_l3_npz, plot_l3_fft_preview

p = Path("artifacts/experiments/cylinder_exp_full/L3_fft/linear_p0004_s0000.npz")

summary = summarize_l3_npz(p)
summary["sanity"]

In [ ]:
figs = plot_l3_fft_preview(p, max_k=40)  # 或不设 max_k
figs

## 1-3级产物一键检查

In [ ]:
from backend.pipelines.train import check_level123_artifacts_ready

rep = check_level123_artifacts_ready(
    "configs/cylinder_exp_small.yaml",
    experiment_name="cylinder_exp_small",
    save_root="artifacts/experiments",
    require_level=3,
    verbose=True,
)

assert rep["ok"]


## 评估指标+出图（4级产物）

In [ ]:
from backend.pipelines.eval.start import compute_level4_eval_mods

pack = compute_level4_eval_mods(
    exp_dir="artifacts/experiments/cylinder_exp_full",
    assemble=[
        "fourier.energy_spectrum_legend",
        "fourier.kstar_heatmap",
        "fourier.band_nrmse_curves",
        "fourier.kstar_curves",
    ],
    mod_kwargs={
        "fourier.kstar_curves": {"max_plots": 40, "show_local_curve": True},
        "fourier.kstar_heatmap": {"use_log10_ell": False},
    },
    verbose=True,
)
pack

In [ ]:
from backend.pipelines.eval.start import compute_level4_eval_mods

pack = compute_level4_eval_mods(
    exp_dir="artifacts/experiments/your_exp",
    assemble=[
        "examples.triptych",
        "examples.fourier_band_decomp",
        "examples.fft2_triptych",
    ]
    mod_kwargs={
        "examples.fft2_triptych": {
            "sample_frames": 4,
            "channel": 0,
            "log_scale": True,
            "dpi": 180,
        }
    },
)
print(pack["fig_paths"].get("examples.fft2_triptych", [])[:3])


# V1 legacy版本控制台

## 线性单帧 Demo

In [ ]:
from backend.pipeline import quick_test_linear_baseline

res = quick_test_linear_baseline(
    nc_path="data/cylinder2d.nc",
    pod_dir="artifacts/pod_r128",
    r=128,
    center=True,
    var_keys=("u", "v"),
    frame_idx=64,
    mask_rate=0.02,
    noise_sigma=0.01,
    max_modes=64,      # 前 64 个模态
    modes_per_fig=16,  # 每 16 个为一组：q1-16, q17-32, q33-48, q49-64
    channel=0,
    verbose=True,
)

print("POD 自身截断误差 NMSE =", res["nmse_pod"])
print("线性基线重建 NMSE      =", res["nmse_linear"])


## MLP 单帧 demo

In [ ]:
from backend.pipeline import quick_test_mlp_baseline

res = quick_test_mlp_baseline(
    nc_path="data/cylinder2d.nc",
    pod_dir="artifacts/pod_r128",
    r=128,
    center=True,
    var_keys=("u", "v"),
    frame_idx=64,
    mask_rate=0.02,
    noise_sigma=0.01,
    mlp_noise_sigma=0.01,
    batch_size=64,
    num_epochs=100,
    lr=1e-3,
    max_modes=128,
    modes_per_fig=16,
    channel=0,
    verbose=True,
)

print("NMSE(POD truncation) =", res["nmse_pod"])
print("NMSE(Linear baseline) =", res["nmse_linear"])
print("NMSE(MLP baseline)   =", res["nmse_mlp"])


## 多模态静态展示

In [ ]:
from backend.dataio.io_utils import load_numpy, load_json
from backend.viz.pod_plots import plot_pod_mode_groups
Ur = load_numpy("artifacts/pod/Ur.npy")
r_eff = min(128, Ur.shape[1])
Ur_eff = Ur[:, :r_eff]
meta = load_json("artifacts/pod/pod_meta.json")
H, W, C = meta["H"], meta["W"], meta["C"]
fig_modes = plot_pod_mode_groups(
    Ur_eff,
    H=H,
    W=W,
    C=C,
    max_modes=128,
    group_size=16,
    channel=0,
)
fig_modes.show()

## 完整 sweep + 多尺度分析

In [ ]:
from backend.pipeline import run_experiment_from_yaml

res = run_experiment_from_yaml(
    "configs/cylinder_exp2.yaml",
    experiment_name="cylinder_exp_legacy",
    save_root="artifacts/experiments",
    generate_report=True,
    verbose=True,
)

print("报告路径:", res["report_path"])
res["df_linear"].head()


## V1绘图控制台

In [ ]:
from backend.viz.field_plots import plot_example_from_npz

fig = plot_example_from_npz(
    "artifacts/experiments/cylinder_exp2_1/p0-0004_sigma0-01/linear_example_01.npz",
)

In [ ]:
from backend.pipeline import redraw_all_example_figures_from_npz
redraw_all_example_figures_from_npz("artifacts/experiments/cylinder_exp4_1")

In [ ]:
from backend.pipeline import quick_figs_from_saved_experiment

figs = quick_figs_from_saved_experiment("artifacts/experiments", "cylinder_exp4_1")
figs.keys()


In [ ]:
figs["fig_fourier_kstar_curves_linear"]

In [ ]:
from pathlib import Path
import numpy as np
npz_path = Path("artifacts/experiments/cylinder_exp_small/p0-0004_sigma0-01/fourier_kstar_curve_linear.npz")
data = np.load(npz_path, allow_pickle=True)
list(data.keys())

In [ ]:
data["cum_meta_json"]

In [ ]:
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt

# 你项目里的函数：按你的实际 import 路径改一下
from backend.viz.field_plots import plot_example_from_npz
from backend.viz.fourier_plots import plot_spatial_fourier_band_decomposition
from backend.dataio.io_utils import ensure_dir  # <- 可能在别的模块里，按实际路径改

# ========= 1) 设定要看的 npz =========
npz_path = Path("artifacts/experiments/cylinder_exp4_1/p0-0004_sigma0/linear_example_00.npz")  # <- 改成你的
exp_root = npz_path.parents[1]

# ========= 2) 读 npz =========
data = np.load(npz_path, allow_pickle=False)
assert {"x_true", "x_hat", "x_interp"}.issubset(set(data.files)), "Not an example NPZ!"

# ========= 3) 四联图（覆盖保存）=========
fig = plot_example_from_npz(npz_path)
png_path = npz_path.with_suffix(".png")
ensure_dir(png_path.parent)
fig.savefig(png_path, dpi=300, bbox_inches="tight")
plt.close(fig)
print("saved:", png_path)

# ========= 4) Fourier 联图（覆盖保存）=========
meta_path = exp_root / "fourier_meta.json"
meta = json.loads(meta_path.read_text(encoding="utf-8"))

band_names = tuple(meta.get("fourier_band_names") or ("L", "M", "H"))
k_edges = [float(v) for v in meta["fourier_k_edges"]]  # interior edges
grid_meta = meta.get("fourier_grid_meta") or {}
dx = float(grid_meta.get("dx", 1.0))
dy = float(grid_meta.get("dy", 1.0))

x_true = np.asarray(data["x_true"])
x_hat = np.asarray(data["x_hat"])

model_type = str(data["model_type"]) if "model_type" in data.files else "model"
p_val = float(data["mask_rate"]) if "mask_rate" in data.files else None
s_val = float(data["noise_sigma"]) if "noise_sigma" in data.files else None
fi = int(data["frame_idx"]) if "frame_idx" in data.files else None

title = f"Fourier bands spatial view ({model_type})"
if p_val is not None and s_val is not None:
    title += f" | p={p_val:.4g}, σ={s_val:.4g}"
if fi is not None:
    title += f" | frame={fi}"

fig_f = plot_spatial_fourier_band_decomposition(
    x_true_hw=x_true,
    x_pred_hw=x_hat,
    k_edges=k_edges,
    band_names=band_names,
    dx=dx,
    dy=dy,
    channel=0,
    title=title,
    center_mode="target_mean",
    robust_q=99.5,
    max_cols=5,
)

png_f = Path(str(npz_path.with_suffix("")) + "_fourier.png")
ensure_dir(png_f.parent)
fig_f.savefig(png_f, dpi=300, bbox_inches="tight")
plt.close(fig_f)
print("saved:", png_f)


In [ ]:
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt

from backend.viz.fourier_plots import plot_kstar_curve_from_entry

exp_dir = Path("artifacts/experiments/cylinder_exp_small")   # 改成你的
cfg = "p0-0004_sigma0"                                 # 改成你的 cfg 文件夹名
npz_path = exp_dir / cfg / "fourier_kstar_curve_linear.npz"

with np.load(npz_path, allow_pickle=True) as z:
    # 把 npz 组装成 plot_kstar_curve_from_entry 期望的 entry 格式
    grid_meta = json.loads(str(z["grid_meta_json"]))
    band_names = [str(x) for x in z["band_names"].tolist()] if z["band_names"].size else ("L","M","H")
    k_edges = z["k_edges"].astype(float).tolist() if z["k_edges"].size else None

    entry = {
        "mask_rate": float(z["mask_rate"]),
        "noise_sigma": float(z["noise_sigma"]),
        "model_name": str(z["model_type"]),
        "fourier_curve": {
            "k_centers": z["k_centers"].astype(float),
            "nrmse_cum": z["nrmse_cum"].astype(float),
            "nrmse_k": z["nrmse_k"].astype(float),
            "k_edges": k_edges,
            "band_names": band_names,
            "k_star": float(z["k_star_cum"]) if np.isfinite(float(z["k_star_cum"])) else None,
            "k_star_threshold": float(z["k_star_threshold_legacy"]) if np.isfinite(float(z["k_star_threshold_legacy"])) else None,
            "grid_meta": grid_meta,
        },
    }

fig = plot_kstar_curve_from_entry(entry)
plt.show()


In [ ]:
entry["fourier_curve"]["k_centers"]

In [ ]:
entry["fourier_curve"]["nrmse_k"]

In [ ]:
entry["fourier_curve"]["nrmse_cum"]

In [ ]:
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt

from backend.eval.reports import load_full_experiment_results
from backend.viz.fourier_plots import plot_kstar_heatmap

exp_dir = Path("artifacts/experiments/cylinder_exp4_1")

loaded = load_full_experiment_results(
    base_dir="artifacts/experiments",
    experiment_name="cylinder_exp4_1"
)

npz_path = next(exp_dir.glob("p*_sigma*/fourier_kstar_curve_linear.npz"))
with np.load(npz_path, allow_pickle=True) as z:
    grid_meta = json.loads(str(z["grid_meta_json"]))

fig = plot_kstar_heatmap(
    loaded["df_linear"],
    loaded.get("df_mlp"),
    model="linear",
    title="Resolvable scale ℓ* heatmap",
    grid_meta=grid_meta,
)

plt.show()
